<a href="https://colab.research.google.com/github/julika333/TMS-DS/blob/main/lesson_25_tensor_and_pytorch/HW_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TensorFlow**
---
---


In [ ]:
!pip install tensorflow keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from keras_tuner import RandomSearch, Hyperband

In [ ]:
# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# Define the model
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Hyperparameter tuning for the number of units in the hidden layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))

    model.add(layers.Dense(10, activation='softmax'))

    # Hyperparameter tuning for the optimizer
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    model.compile(optimizer=hp_optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Random search tuner
random_search_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='random_search_dir',
    project_name='mnist_tuning')

# Hyperband tuner
hyperband_tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    directory='hyperband_dir',
    project_name='mnist_tuning')

# Train the models
random_search_tuner.search(x_train, y_train, epochs=5, validation_split=0.2)
hyperband_tuner.search(x_train, y_train, epochs=5, validation_split=0.2)

# Get the best model
best_model = random_search_tuner.get_best_models(num_models=1)[0]
best_model.summary()


Trial 30 Complete [00h 02m 23s]
val_accuracy: 0.9792500138282776

Best val_accuracy So Far: 0.9807500243186951
Total elapsed time: 00h 19m 45s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 384)               301440    
                                                                 
 dense_1 (Dense)             (None, 10)                3850      
                                                                 
Total params: 305290 (1.16 MB)
Trainable params: 305290 (1.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
hyperband_tuner.results_summary()


Results summary
Results in hyperband_dir/mnist_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0017 summary
Hyperparameters:
units: 320
optimizer: rmsprop
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.9807500243186951

Trial 0029 summary
Hyperparameters:
units: 512
optimizer: adam
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9792500138282776

Trial 0023 summary
Hyperparameters:
units: 256
optimizer: rmsprop
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9781666398048401

Trial 0024 summary
Hyperparameters:
units: 256
optimizer: rmsprop
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 0.9781666398048401

Trial 0016 summary
Hyperparameters:
units: 480
optimizer: adam
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.97791665792

# **PyTorch**
---
---


In [ ]:
!pip install torch torchvision torch-summary optuna

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import optuna
from optuna.trial import TrialState

In [ ]:
# Load dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST('.', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('.', train=False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define the model
class Net(nn.Module):
    def __init__(self, trial):
        super(Net, self).__init__()
        n_units = trial.suggest_int("n_units", 32, 512)
        self.fc1 = nn.Linear(28 * 28, n_units)
        self.fc2 = nn.Linear(n_units, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# Define the objective function
def objective(trial):
    model = Net(trial).to(device)

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    criterion = nn.CrossEntropyLoss()

    # Training
    for epoch in range(5):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / len(test_loader.dataset)
    return accuracy

# Use Optuna for hyperparameter optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Get the best trial
trial = study.best_trial

print(f"Best trial: {trial.values}")
print(f"Best hyperparameters: {trial.params}")

[I 2024-05-23 21:20:44,297] A new study created in memory with name: no-name-d203bba5-202c-4fe1-a82b-873c82118b87
[I 2024-05-23 21:21:54,973] Trial 0 finished with value: 0.8983 and parameters: {'n_units': 171, 'optimizer': 'SGD', 'lr': 0.0023356773941556976}. Best is trial 0 with value: 0.8983.
[I 2024-05-23 21:23:04,060] Trial 1 finished with value: 0.9037 and parameters: {'n_units': 135, 'optimizer': 'SGD', 'lr': 0.0031119474959408664}. Best is trial 1 with value: 0.9037.
[I 2024-05-23 21:24:13,011] Trial 2 finished with value: 0.2338 and parameters: {'n_units': 70, 'optimizer': 'SGD', 'lr': 2.0135904531693266e-05}. Best is trial 1 with value: 0.9037.
[I 2024-05-23 21:25:23,392] Trial 3 finished with value: 0.765 and parameters: {'n_units': 370, 'optimizer': 'SGD', 'lr': 0.00020980772035515514}. Best is trial 1 with value: 0.9037.
[I 2024-05-23 21:26:34,391] Trial 4 finished with value: 0.9726 and parameters: {'n_units': 263, 'optimizer': 'RMSprop', 'lr': 0.0010489875047035024}. Bes

Best trial: [0.9726]
Best hyperparameters: {'n_units': 263, 'optimizer': 'RMSprop', 'lr': 0.0010489875047035024}


In [ ]:
# PyTorch Notebook

# Install necessary packages
# !pip install torch torchvision torchsummary optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import optuna
from torchsummary import summary

# Load dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model
class MNISTModel(nn.Module):
    def __init__(self, units):
        super(MNISTModel, self).__init__()
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(28*28, units)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(units, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.softmax(x)
        return x

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter tuning for the number of units in the hidden layer
    units = trial.suggest_int('units', 32, 512, step=32)
    model = MNISTModel(units)

    # Hyperparameter tuning for the optimizer
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop', 'sgd'])
    optimizer = getattr(optim, optimizer_name.capitalize())(model.parameters(), lr=0.01)

    criterion = nn.CrossEntropyLoss()

    # Training the model
    model.train()
    for epoch in range(5):
        for batch in train_loader:
            inputs, targets = batch
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Validation accuracy
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == targets).sum().item()

    accuracy = correct / len(test_dataset)
    return accuracy

# Random search tuner
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Get the best model
trial = study.best_trial
best_units = trial.params['units']
best_model = MNISTModel(best_units)

# Summary of the best model
summary(best_model, input_size=(1, 28, 28))


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 17393300.77it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 477323.73it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5259671.11it/s]
[I 2024-05-23 21:45:58,536] A new study created in memory with name: no-name-41cd7765-485b-4f8d-8d2f-395b69d8afe7


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



[I 2024-05-23 21:47:14,318] Trial 0 finished with value: 0.8077 and parameters: {'units': 64, 'optimizer': 'adam'}. Best is trial 0 with value: 0.8077.
[I 2024-05-23 21:48:51,806] Trial 1 finished with value: 0.101 and parameters: {'units': 384, 'optimizer': 'adam'}. Best is trial 0 with value: 0.8077.
[W 2024-05-23 21:48:51,813] Trial 2 failed with parameters: {'units': 128, 'optimizer': 'sgd'} because of the following error: AttributeError("module 'torch.optim' has no attribute 'Sgd'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-11-317b8838e6be>", line 48, in objective
    optimizer = getattr(optim, optimizer_name.capitalize())(model.parameters(), lr=0.01)
AttributeError: module 'torch.optim' has no attribute 'Sgd'
[W 2024-05-23 21:48:51,814] Trial 2 failed with value None.


AttributeError: module 'torch.optim' has no attribute 'Sgd'